In [7]:
from __future__ import annotations
import ctypes
from ctypes import *
import os
import sys
import subprocess
import numpy as np
from PIL import Image
import scipy.stats as st
sys.path.append(sys.path[0]+"/..")
from SYS_ATL import proc, Procedure
sys.path.append(sys.path[0]+".")
from helper import *
from pygments.styles import get_all_styles
from pygments.lexers import PythonLexer
from pygments.formatters import Terminal256Formatter
from pygments import highlight

def pprint(code):
    highlighted_code = highlight(str(code), PythonLexer(), Terminal256Formatter(style='default'))
    print(highlighted_code)

In [8]:
def gen_blur():
    @proc
    def blur(n: size, m: size, k_size: size,
             image: R[n, m] @ IN, kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
        for i in par(0, n):
            for j in par(0, m):
                res[i, j] = 0.0
        for i in par(0, n):
            for j in par(0, m):
                for k in par(0, k_size):
                    for l in par(0, k_size):
                        if i+k >= 1 and i+k-n < 1 and j+l >= 1 and j+l-m < 1:
                            res[i, j] += kernel[k, l] * image[i+k-1, j+l-1]

    return blur

In [9]:
blur = gen_blur()

In [10]:
pprint(blur)

def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i in par(0, n):
        for j in par(0, m):
            res[i, j] = 0.0
    for i in par(0, n):
        for j in par(0, m):
            for k in par(0, k_size):
                for l in par(0, k_size):
                    if i + k >= 1 and i + k - n < 1 and j + l >= 1 and j + l - m < 1:
                        res[i, j] += kernel[k, l] * image[i + k - 1, j + l - 1]



In [11]:
pprint(blur.reorder('k', 'l'))

def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i in par(0, n):
        for j in par(0, m):
            res[i, j] = 0.0
    for i in par(0, n):
        for j in par(0, m):
            for l in par(0, k_size):
                for k in par(0, k_size):
                    if i + k >= 1 and i + k - n < 1 and j + l >= 1 and j + l - m < 1:
                        res[i, j] += kernel[k, l] * image[i + k - 1, j + l - 1]



In [12]:
pprint(blur.split('j',4,['j1','j2']))

def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i in par(0, n):
        for j1 in par(0, m / 4):
            for j2 in par(0, 4):
                if 4 * j1 + j2 < m:
                    res[i, 4 * j1 + j2] = 0.0
    for i in par(0, n):
        for j1 in par(0, m / 4):
            for j2 in par(0, 4):
                if 4 * j1 + j2 < m:
                    for k in par(0, k_size):
                        for l in par(0, k_size):
                            if i + k >= 1 and i + k - n < 1 and 4 * j1 + j2 + l >= 1 and 4 * j1 + j2 + l - m < 1:
                                res[i, 4 * j1 +
                                    j2] += kernel[k, l] * image[i + k - 1,
                                                                4 * j1 + j2 +
                                                                l - 1]



In [13]:
pprint(blur.split('i[1]',4,['i1','i2']))

def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i1 in par(0, n / 4):
        for i2 in par(0, 4):
            if 4 * i1 + i2 < n:
                for j in par(0, m):
                    res[4 * i1 + i2, j] = 0.0
    for i in par(0, n):
        for j in par(0, m):
            for k in par(0, k_size):
                for l in par(0, k_size):
                    if i + k >= 1 and i + k - n < 1 and j + l >= 1 and j + l - m < 1:
                        res[i, j] += kernel[k, l] * image[i + k - 1, j + l - 1]



In [14]:
blur = blur.split('j',4,['j1','j2'])
blur = blur.unroll('j2')
pprint(blur)

def blur(n: size, m: size, k_size: size, image: R[n, m] @ IN,
         kernel: R[k_size, k_size] @ IN, res: R[n, m] @ OUT):
    for i in par(0, n):
        for j1 in par(0, m / 4):
            if 4 * j1 + 0 < m:
                res[i, 4 * j1 + 0] = 0.0
            if 4 * j1 + 1 < m:
                res[i, 4 * j1 + 1] = 0.0
            if 4 * j1 + 2 < m:
                res[i, 4 * j1 + 2] = 0.0
            if 4 * j1 + 3 < m:
                res[i, 4 * j1 + 3] = 0.0
    for i in par(0, n):
        for j1 in par(0, m / 4):
            if 4 * j1 + 0 < m:
                for k in par(0, k_size):
                    for l in par(0, k_size):
                        if i + k >= 1 and i + k - n < 1 and 4 * j1 + 0 + l >= 1 and 4 * j1 + 0 + l - m < 1:
                            res[i, 4 * j1 +
                                0] += kernel[k, l] * image[i + k - 1,
                                                           4 * j1 + 0 + l - 1]
            if 4 * j1 + 1 < m:
                for k in 